In [ ]:
import pandas as pd
import duckdb as ddb
import glob
import os
import re
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import numpy as np

In [ ]:
log_datapath= "G:/Innovations@HELLER/DN/KI/Temperaturkompensation/2021_Spindelwachstumskompensation_KI_HSU_SC63/Messungen DC100 H5000 M57002/data _from_external_device/CSV-Dateien/"
orginal_data= "G:/Innovations@HELLER/DN/KI/Temperaturkompensation/2021_Spindelwachstumskompensation_KI_HSU_SC63/Messungen DC100 H5000 M57002/row_files/combines/smoothed/With_cooling_feature_for_Model_training_5s/validate/validate_Data/"
temp_file=''
log_file= ''
log_selected_columns= ['t_kss_tank', 't_raum', 't_spindel_rueck', 't_spindel_vor', 'volumenstrom', 'v_maschienenkuehler', 't_z_achse' ]#'t_y_achse', 'ch9', 'ch20

In [ ]:
temp_data_date_format= "%Y-%m-%d %H:%M:%S.%f" #"%Y.%m.%d %H:%M:%S"# "2022-02-25 15:30:00"
df_logs_date_format= "%Y-%m-%d %H:%M:%S"

In [ ]:
def dateparse (time_str):    
    return pd.to_datetime(time_str)

In [ ]:
def handle_special_numbers(df: pd.DataFrame, columns_list: list):
    for column in columns_list:
        new_column= df[column].str.replace(pat='\+( )+',repl= '+',regex= True).str.replace(pat= '-( )+', repl= '-',regex= True)
        df[column]= pd.to_numeric(new_column)
    return df

In [ ]:
def create_block(start_date:datetime, period: int, count_insertions: int, features: list):
    sample_dict= {'date': [ dateparse(start_date) + timedelta(seconds= i*period) for i in range(1,count_insertions + 1)]}
    for column in features:
        sample_dict[column]= [np.NaN for i in range(count_insertions)] 
    #insertion_block= pd.DataFrame({'date': [ datetime.strptime(start_date,"%Y-%m-%d %H:%M:%S") + timedelta(seconds= i*period) for i in range(1,count_insertions + 1)],'t_bett': [np.NaN for i in range(count_insertions)],'t_motor':[np.NaN for i in range(count_insertions)], 't_spindel': [np.NaN for i in range(count_insertions)],'z_welle_ok':[np.NaN for i in range(count_insertions)]})
    insertion_block= pd.DataFrame(sample_dict)
    
    return insertion_block

In [ ]:
def insert_block(original_data: pd.DataFrame, period:int,  count_insertions: int, features):
    output_df= None
    for index, row in original_data.iterrows():
        if index + 1 == len(original_data):
            output_df= pd.concat([output_df,row.to_frame().T])
        else:
            start_date= row['date']
            block= create_block(start_date= start_date, period= period, count_insertions= count_insertions, features= features)
            #print('block', block)
            if index == 0:
                output_df= pd.concat([row.to_frame().T, block], axis=0, ignore_index= True)
            else:
                output_df= pd.concat([output_df, row.to_frame().T, block], axis=0, ignore_index= True)
        #print(output_df[['date', 't_spindel_vor']])
    return output_df


Load Sensor Files in one Data Frame and insert Nan blocks to prepare for the interpolation

In [ ]:
working_dir= str(log_datapath)
os.chdir(working_dir)
result_df= None
for file in glob.glob('*.CSV'):
    print ('Current File: ',file)
    content = open(file).readlines()
    lines = [line_num for line_num, line_content in enumerate(content) if "Data" in line_content]
    skipped_rows= list(range(0,lines[0]+1))
    skipped_rows.append(lines[0]+2)
    df_logs= pd.read_csv(file, sep=',',skiprows= skipped_rows, infer_datetime_format= True, parse_dates= True, date_parser= dateparse)#,index_col=['Date&Time'])
    df_logs.drop(columns=["Alarm1-10","Alarm11-20","AlarmOut","ms","Number"], inplace=True)
    df_logs.rename(columns={'Date&Time':'date','CH1': 'T_KSS_TANK', 'CH2':'T_RAUM', 'CH3':'T_SPINDEL_RUECK', 'CH4':'T_SPINDEL_Vor', 'CH5':'Volumenstrom','CH6':'V_MASCHIENENKUEHLER','CH8':'T_Z_ACHSE'}, inplace= True)# 'CH7':'T_Y_ACHSE',
    df_logs.columns= df_logs.columns.str.lower()
    dates= df_logs['date']
    df_logs= df_logs[log_selected_columns]
    df_logs['date']= dates
    df_logs= handle_special_numbers(df_logs, columns_list=['t_kss_tank', 't_raum', 't_spindel_vor','t_spindel_rueck', 'volumenstrom', 'v_maschienenkuehler', 't_z_achse'])
    df_logs['t_spindel_rueck'] += 0.2
    ###Upsample up to 1 second
    df_logs= insert_block(df_logs, period= 1, count_insertions=19, features= log_selected_columns)
    if result_df is None:
        result_df= df_logs
    else:
        result_df= pd.concat([result_df, df_logs])
    print('Current Size=',  len(result_df))
df_logs= result_df
df_logs_fields= df_logs.columns.to_list()
print (df_logs_fields)
print(df_logs.head(40))


In [ ]:
df_logs= result_df.copy()

Create a dictionary of the features and their required types

In [ ]:
float_dict= { column:'float' for column in log_selected_columns}
float_dict

Set feature types as float

In [ ]:
df_logs= df_logs.astype(float_dict)

Apply interpolation

In [ ]:
df_logs_interpolated = df_logs.interpolate(method='linear', order=3, axis= 0,inplace= False)

In [ ]:
df_logs_interpolated.info()

In [ ]:
df_logs_interpolated.to_csv(str(log_datapath+'combinded_interpolated_1s.csv'))

load files from other datasource

In [ ]:
log_datapath= "G:/Innovations@HELLER/DN/KI/Temperaturkompensation/2021_Spindelwachstumskompensation_KI_HSU_SC63/Messungen DC100 H5000 M57002/data _from_external_device/integration/"
df_logs= pd.read_csv(str(log_datapath+'combinded_interpolated_1s.csv'), infer_datetime_format= True, parse_dates= True, date_parser= dateparse)#pd.read_csv(filepath_or_buffer= str(log_datapath+'combinded_interpolated_1s.csv'))
temp_data= pd.read_csv(str(log_datapath+'all_data.csv'), infer_datetime_format= True, parse_dates= True, date_parser= dateparse)#pd.read_csv(filepath_or_buffer= str(log_datapath+'all_data.csv'))
df_logs= df_logs.drop(columns=['Unnamed: 0'])
# df_logs['date']= datetime.strftime(df_logs['date'],format= df_logs_date_format)
# df_logs['date']= datetime.strptime(df_logs['date'],format= df_logs_date_format)

Create a part of the required features in the query

In [ ]:
def get_field_list(df: pd.DataFrame, df_name: str):
    df_logs_fields= df.columns.to_list()
    fields= ""
    for field in df_logs_fields:
        if fields == "":
            fields =  "{df_name}.".format(df_name= df_name) + field
        else:
            fields += ", {df_name}.".format(df_name= df_name) + field
    return fields

Create a list of features for each datasource

In [ ]:
df_logs_fields=get_field_list(df_logs, "df_logs")
temp_data_fields=get_field_list(temp_data, "temp_data")

In [ ]:
df_logs_fields

In [ ]:
temp_data_fields

Apply the query to aggregate the two data sources

In [ ]:
####Apply Query
query= """
select * from 
    (select temp_data.start as start, temp_data.end as end,{temp_data_fields}, min(df_logs.ts)  as ts from 
        (
        (select  {temp_data_fields}, strptime(temp_data.date,'{temp_data_date_format}') as start, (strptime(temp_data.date,'{temp_data_date_format}') + interval 5 Second) as end   from temp_data) as temp_data
        join
        (select {df_logs_fields}, strptime(df_logs.date ,'{df_logs_date_format}') as ts from df_logs) as df_logs
        on (df_logs.ts >= temp_data.start) and ( df_logs.ts < temp_data.end)
        )
    group by temp_data.start, temp_data.end, {temp_data_fields}) combined_logs
join
    df_logs
on (combined_logs.ts = strptime(df_logs.date ,'{df_logs_date_format}'))
order by start Asc
""".format(temp_data_fields= temp_data_fields, df_logs_fields= df_logs_fields,
            temp_data_date_format= temp_data_date_format, df_logs_date_format= df_logs_date_format)
results= ddb.query(query)
result_df= results.to_df()

Check the correctness of aggregation based on time

In [ ]:
result_df[['start','date', 'end', 'ts', 'date_2']]

In [ ]:
result_df.info()

Remove doubled columns and save the data

In [ ]:
new_df= result_df.drop(columns=['start', 'end', 'ts'])
new_df.to_csv(str(log_datapath+'aggregated_data.csv'))

In [ ]:
new_df.info()

In [ ]:
import seaborn as sn

In [ ]:
correlations= new_df.corr()
fig, ax = plt.subplots(figsize=(15, 15))
plt.title(' Heat-Map für die Korrelationsmatrix')
ax= sn.heatmap(correlations, annot=True, vmin=-1, vmax=1, cmap='magma', annot_kws={"size": 10, 'color': 'black'})
#plt.savefig(images_analysis_path.format(data_mode,'row_data_corr.png') )
plt.show()

In [ ]:
min_nr= new_df['experiment_nr'].min()
max_nr= new_df['experiment_nr'].max()

In [ ]:
for i in range(min_nr, max_nr+1):
    to_save_df= new_df[new_df['experiment_nr']== i]
    if (len(to_save_df) > 0):
        to_save_df.to_csv(str(log_datapath+ str(i)+'.csv'))

In [ ]:
to_save_df.columns.tolist()

In [ ]:
selected_columns= [
 't_bett',
 't_motor',
 't_spindle',
 'DRZ2',
 'M8',
 'M121',
 'M127',
 'M7',
 'welle_z_iterpolated',
 't_kss_tank',
 't_raum',
 't_spindel_rueck',
 't_spindel_vor',
 'volumenstrom',
 'v_maschienenkuehler',
 't_z_achse']

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as ex
import plotly.io as pio

In [ ]:
for i in range(min_nr, max_nr+1):
    to_save_df= new_df[new_df['experiment_nr']== i]
    if (len(to_save_df) > 0):
        fig= make_subplots(rows=len(selected_columns) ,cols=1,shared_xaxes= True, print_grid= True, subplot_titles= selected_columns, vertical_spacing=0.02)
        for j in range(len(selected_columns)):
            fig.add_trace(go.Scatter(x= to_save_df['date'], y= to_save_df[selected_columns[j]], name=selected_columns[j], mode= 'lines'), row= j+1, col= 1)
            fig.update_yaxes(title_text= selected_columns[j], row= i+1, col= 1)
        fig.update_layout(height=900, width=1200, title_text=str(i))
        fig.show()